In [ ]:
import torch
from diffusers import AutoencoderKL
from tqdm.notebook import tqdm
import numpy as np
from transformer.microdit import ReiMei, ReiMeiParameters
import matplotlib.pyplot as plt
from config import VAE_CHANNELS, DIT_S as DIT, MODELS_DIR_BASE
from torch.amp import autocast
from open_clip import create_model_from_pretrained, get_tokenizer
from diffusers import AutoencoderDC

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
device

In [4]:
base_dim = 1024
base_heads = 16

input_dim = VAE_CHANNELS
embed_dim = 1024
num_layers = 28
num_heads = 16
mlp_dim = embed_dim
cond_embed_dim = 1 # Null for this dataset
# pos_embed_dim = 60
pos_embed_dim = None
num_experts = 32
active_experts = 2.0
shared_experts = None
token_mixer_layers = 2
dropout = 0.1

m_d = float(embed_dim) / float(base_dim)

assert (embed_dim // num_heads) == (base_dim // base_heads)

params = ReiMeiParameters(
    channels=input_dim,
    embed_dim=embed_dim,
    num_layers=num_layers,
    num_heads=num_heads,
    mlp_dim=mlp_dim,
    text_embed_dim=cond_embed_dim,
    vector_embed_dim=cond_embed_dim,
    num_experts=num_experts,
    active_experts=active_experts,
    shared_experts=shared_experts,
    dropout=dropout,
    token_mixer_layers=token_mixer_layers,
    m_d=m_d,
)
DTYPE = torch.bfloat16

In [ ]:
model = ReiMei(params)
# model.load_state_dict(torch.load("models/microdit_model_and_optimizer_epoch_40_f32.pt")['model_state_dict'])

In [6]:
model = model.to(device).to(torch.bfloat16)

In [ ]:
# Print the number of parameters in the model
print("Number of parameters in the model: ", sum(p.numel() for p in model.parameters()))


In [ ]:
vae = AutoencoderDC.from_pretrained("mit-han-lab/dc-ae-f32c32-in-1.0-diffusers", torch_dtype=DTYPE, cache_dir=f"{MODELS_DIR_BASE}/dc_ae", revision="main").to(device, ).eval()
# print param count
print(sum(p.numel() for p in vae.parameters()))

In [9]:
# SIGLIP_HF_NAME ="hf-hub:timm/ViT-SO400M-14-SigLIP-384"

In [10]:
# siglip_model, _ = create_model_from_pretrained(SIGLIP_HF_NAME, precision="fp16", cache_dir=f"{MODELS_DIR_BASE}/siglip")
# siglip_model = siglip_model.to(device)
# siglip_tokenizer = get_tokenizer(SIGLIP_HF_NAME)

In [11]:
# prompts = [
#     "A serene landscape with rolling hills under a vibrant sunset sky",
#     "A futuristic cityscape with towering skyscrapers and flying cars",
#     "A cozy cottage in a snowy forest with smoke coming from the chimney"
# ]

In [12]:
# prompts = [
#     "amateur photography, a woman showing the palms of her hands",
#     "A futuristic cityscape with towering skyscrapers and flying cars, where the buildings are adorned with neon lights and digital billboards, creating a dazzling display of colors against the night sky, while sleek, aerodynamic vehicles zip through the air, leaving trails of light in their wake.",
#     "A cozy cottage in a snowy forest, with smoke gently rising from the chimney, surrounded by tall pine trees laden with fresh snow, their branches glistening in the soft glow of the moonlight, as the warm light from the cottage windows spills out onto the snow-covered ground, inviting and comforting."
# ]

In [13]:
# texts = siglip_tokenizer(prompts, context_length=siglip_model.context_length).to(device)
# caption_embeddings = siglip_model.encode_text(texts)

In [14]:
caption_embeddings = torch.zeros((1, cond_embed_dim), device=device, dtype=DTYPE)

In [18]:
noise = torch.randn(1, 32, 16, 16).to(device).to(torch.bfloat16)


In [ ]:
model = model.eval()
with autocast("cuda", dtype=torch.bfloat16):
    pred = model.sample(noise, caption_embeddings.unsqueeze(1), caption_embeddings, sample_steps=50, cfg=7.0)

    pred = vae.decode(pred).sample
    # Change range of pred from x to y to -1 to 1
    min_val = pred.min()
    max_val = pred.max()

    pred = (pred - min_val) / (max_val - min_val)
    pred = 2 * pred - 1


with torch.inference_mode():
    pred_cpu = pred.cpu().to(torch.float32)
    pred_np = pred_cpu.permute(0, 2, 3, 1).numpy()
    pred_np = (pred_np + 1) / 2
    pred_np = (pred_np * 255).astype(np.uint8)  # Convert to uint8

    # Create figure for predictions
    fig_pred, axes_pred = plt.subplots(3, 3, figsize=(12, 12))  # 3 rows, 3 columns

    # Plot predictions
    for i, ax in enumerate(axes_pred.flatten()):
        if i < pred_np.shape[0]:
            ax.imshow(pred_np[i])
            # ax.set_title(prompts[i])  # Add this line to set the title
            ax.axis('off')
        else:
            ax.axis('off')
    
    plt.tight_layout()
    plt.show()